In [1]:
import os
import numpy as np
from nibabel.testing import data_path
import nibabel as nib

import keras
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
%matplotlib inline
from IPython.display import display


Using TensorFlow backend.


In [20]:
import nilearn
from nilearn import plotting
from nilearn import image
from nilearn import datasets

In [21]:
# Get dataset with nilearn function
# if not downloaded, will download. If already downloaded, will uses local version
dataset = nilearn.datasets.fetch_cobre(n_subjects=146, data_dir="/Users/KJP/Desktop/neural nets/Cobre Dataset2", url=None, verbose=1)


In [97]:
phenotypes = dataset["phenotypic"]
file_paths = dataset["func"]
phenotypes.sort(0) #sort by column corresponding to patient number
file_paths.sort() #sort file names by alphabetical order, which will result in sorting by patient number
display(phenotypes[0])
display(file_paths[0])
#file_paths is now a regular python list of the file paths to the fmri scans
#phenotypes is now a np.recarray of np.records storing patient info

(40000, 20, b'Female', b'Right', b'Patient', b'295.9', 140, 0.21234, 0.20245)

'/Users/KJP/Desktop/neural nets/Cobre Dataset2/cobre/fmri_0040000.nii.gz'

list

In [98]:
# get just the diagnosis information from the phenotypes
diagnosis = phenotypes['diagnosis']
diagnosis_converted = []
#this stem is necessary to convert np.byte array into strings, and then fit those strings into 2 categories:
#Schizophrenia or no Schizophrenia
for item in diagnosis:
    s = item.decode('UTF-8')
    if s != "None":
        diagnosis_converted.append(float(1))   #person has schizophrenia
    else:
        diagnosis_converted.append(float(0))   #person doesn't have schizophrenia

del diagnosis_converted[74]                  # item 74 is a messed up scan with different dimensions
del file_paths[74]                           # so it needs to be removed

y_train = np.array(diagnosis_converted)
y_train = keras.utils.to_categorical(y_train, len(np.unique(y_train))) #one hot encoding

In [108]:
# Make x train from the file paths
scans = []
for item in file_paths:
    scan = nib.load(item)
    data = scan.get_fdata()
    scans.append(data)
x_train = np.array(scans)

In [112]:
#X train is now 145 different fmri scans, with dimensions 27x32x26x150
#the 27x32x26 is length width and height
#the 150 is time, there are 150 different 3d 'voxels' or times for each full fmri scan
x_train.shape

(145, 27, 32, 26, 150)

In [113]:
#Y train is simply 145 numpy arrays of either [0,1] or [1,0], representing whether the patient has schizophrenia or not
y_train.shape


(145, 2)

In [117]:
np.save('/Users/KJP/Desktop/neural nets/Codes/XData.npy', x_train)
np.save('/Users/KJP/Desktop/neural nets/Codes/YData.npy', y_train)

